In [6]:
# === Restart Kernel First ===
# Ensure a clean environment

import os
import sys
import pandas as pd
from pathlib import Path
from IPython.display import display
from dotenv import load_dotenv

# === Correct project root on server ===
project_root = Path("/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission")

# Add project directory to sys.path if needed
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Debug: confirm config.py exists
print("🔍 config.py exists:", (project_root / "config.py").exists())

# Load .env from the correct location
load_dotenv(dotenv_path=project_root / ".env")

# Now import
from config import load_cleaned_data

# Load Data
df = load_cleaned_data()

# Preview
display(df.head())


# === Rename for internal use ===
df = df.rename(columns={
    "UniqueIncidentKey": "incident_id",
    "Patient Age (ePatient.15)": "age",
    "Patient Age Units (ePatient.16)": "age_units",
    "Primary Impression": "primary_impression",
    "Secondary Impression": "secondary_impression",
    "Transport Disposition": "transport_disposition",
    "Stroke Alert": "stroke_alert",
    "Situation Last Known Well Date Time (eSituation.18)": "lkw_time",
    "Situation Symptom Onset Date Time (eSituation.01)": "onset_time",
    "Vitals Signs Taken Date Time (eVitals.01)": "vitals_time",
    "Cardiac Arrest During EMS Event With Code (eArrest.01)": "cardiac_arrest",
    "Disposition Final Patient Acuity Code (eDisposition.19)": "final_acuity",
    "Response Type Of Service Requested With Code (eResponse.05)": "response_type"
})

# === Convert time columns ===
df["lkw_time"] = pd.to_datetime(df["lkw_time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
df["onset_time"] = pd.to_datetime(df["onset_time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")
df["vitals_time"] = pd.to_datetime(df["vitals_time"], format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

# === Extract ICDs ===
def extract_icd_prefix(text):
    if isinstance(text, str):
        match = re.search(r"\(([A-Z]\d{2}(?:\.\d+)?)\)", text.upper())
        if match:
            return match.group(1)[:3]
    return ""

df["primary_icd"] = df["primary_impression"].apply(extract_icd_prefix)
df["secondary_icd"] = df["secondary_impression"].apply(extract_icd_prefix)

# === Aggregate by incident ===
grouped = df.groupby("incident_id").agg({
    "age": "first",
    "age_units": "first",
    "primary_icd": "first",
    "secondary_icd": "first",
    "transport_disposition": "first",
    "response_type": "first",
    "vitals_time": "min",
    "lkw_time": "first",
    "onset_time": "first",
    "cardiac_arrest": "first",
    "final_acuity": "first",
    "stroke_alert": lambda x: any(str(val).strip().lower() == "true" for val in x)
}).reset_index()

# === Quarter Extraction ===
grouped["quarter"] = grouped["vitals_time"].dt.to_period("Q")

# === Logic filters ===
grouped["age"] = pd.to_numeric(grouped["age"], errors="coerce")
age_valid = grouped["age"] >= 18
stroke_icds = ["I60", "I61", "I62", "I63", "G45", "G46"]
impression_valid = grouped["primary_icd"].isin(stroke_icds) | grouped["secondary_icd"].isin(stroke_icds)
transport_valid = grouped["transport_disposition"].str.contains("transport by this ems unit", case=False, na=False)
response_valid = grouped["response_type"].str.contains("2205001", na=False)

# === LKW vs onset fallback ===
grouped["time_reference"] = grouped["lkw_time"]
grouped.loc[grouped["time_reference"].isna(), "time_reference"] = grouped["onset_time"]

lkw_exclude = (
    grouped["time_reference"].notna() & grouped["vitals_time"].notna() &
    ((grouped["vitals_time"] - grouped["time_reference"]).dt.total_seconds() >= 86400)
)

arrest_exclude = grouped["cardiac_arrest"].astype(str).isin(["3001003", "3001005"])
acuity_exclude = grouped["final_acuity"].astype(str) == "4219909"

grouped["in_denominator"] = (
    age_valid & impression_valid & transport_valid & response_valid &
    ~lkw_exclude & ~arrest_exclude & ~acuity_exclude
)

grouped["in_numerator"] = grouped["in_denominator"] & grouped["stroke_alert"]

# === Summary ===
summary = (
    grouped[grouped["in_denominator"]]
    .groupby("quarter")
    .agg(
        AHAEMS1_Denominator=("in_denominator", "sum"),
        AHAEMS1_Numerator=("in_numerator", "sum")
    )
    .reset_index()
)
summary["AHAEMS1_Percentage"] = (summary["AHAEMS1_Numerator"] / summary["AHAEMS1_Denominator"] * 100).round(2)


display(summary)

# === Plot AHA Measure 1 Line Graph ===
plt.figure(figsize=(10, 5))
plt.plot(summary["quarter"].astype(str), summary["AHAEMS1_Percentage"], marker="o", linestyle="-", label="AHAEMS1 %")
plt.title("AHA EMS Measure 1 – Stroke Alerts Over Time")
plt.xlabel("Quarter")
plt.ylabel("Percentage (%)")
plt.ylim(0, 100)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# === Export Fallout CSV ===
fallouts = grouped[grouped["in_denominator"] & ~grouped["in_numerator"]]
fallout_path = FALLOUTS_DIR / "ahaems1_fallouts.csv"

# Ensure directory exists before writing
os.makedirs(fallout_path.parent, exist_ok=True)

# Write fallout file
fallouts.to_csv(fallout_path, index=False)

from project_paths import OUTPUT_DIR

# Save the summary chart to output/charts/
chart_path = OUTPUT_DIR / "charts" / "ahaems1_chart.png"
os.makedirs(chart_path.parent, exist_ok=True)
plt.savefig(chart_path, bbox_inches="tight")

🔍 config.py exists: False


ModuleNotFoundError: No module named 'config'

In [7]:
import os
project_root = "/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission"
print("🔍 config.py exists:", os.path.exists(os.path.join(project_root, "config.py")))


🔍 config.py exists: False


In [8]:
import sys
from pathlib import Path

project_root = Path("/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("✅ sys.path includes project:", str(project_root) in sys.path)
print("🔍 Checking config.py:", (project_root / "config.py").exists())

import os
print("📁 Current working directory:", os.getcwd())


✅ sys.path includes project: True
🔍 Checking config.py: False
📁 Current working directory: /home/jovyan


In [5]:
!ls /mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission


ls: cannot access '/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission': No such file or directory


In [9]:
import importlib.util

config_path = project_root / "config.py"
print("🔍 config.py exists:", config_path.exists())
print("📁 Current working directory:", os.getcwd())
print("✅ sys.path includes project:", str(project_root) in sys.path)

# Optional: load config manually to confirm visibility
spec = importlib.util.spec_from_file_location("config", config_path)
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

df = config.load_cleaned_data()

🔍 config.py exists: False
📁 Current working directory: /home/jovyan
✅ sys.path includes project: True


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission/config.py'

In [10]:
!ls -lah /mnt/nasdrive/jupyter/EMS_QI_Projects/

ls: cannot access '/mnt/nasdrive/jupyter/EMS_QI_Projects/': No such file or directory


In [11]:
import os
print("🧠 Kernel working directory:", os.getcwd())


🧠 Kernel working directory: /home/jovyan


In [2]:
import sys
from pathlib import Path

# Project path where config.py is located
project_root = Path("/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission")

# Confirm config.py exists
print("✅ config.py exists:", (project_root / "config.py").exists())

# Print current working directory
print("📍 Working dir:", Path.cwd())

# Add project root to sys.path (if not already)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # insert at front

# Show Python path for debugging
print("🔍 sys.path:")
for p in sys.path:
    print("   ", p)

✅ config.py exists: False
📍 Working dir: /home/jovyan
🔍 sys.path:
    /home/jovyan/undefined-pycharm-support-libs/jupyter_debug
    /home/jovyan/undefined-pycharm-support-libs/pydev
    /opt/conda/lib/python311.zip
    /opt/conda/lib/python3.11
    /opt/conda/lib/python3.11/lib-dynload
    
    /home/jovyan/.venvs/ahaems/lib/python3.11/site-packages
    /mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission


In [3]:
import os

project_dir = "/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission"
print("📁 Directory contents:")
print(os.listdir(project_dir))

📁 Directory contents:


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission'

In [4]:
ls -l /mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission/config.py

ls: cannot access '/mnt/nasdrive/jupyter/EMS_QI_Projects/ahaems-2025-submission/config.py': No such file or directory


In [6]:
from pathlib import Path
import sys

# Adjust project root to match container mount
project_root = Path("/home/jovyan/work/ahaems-2025-submission")

# Add to sys.path for module access
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("✅ config.py exists:", (project_root / "config.py").exists())

# Import function
from config import load_cleaned_data

✅ config.py exists: False


ModuleNotFoundError: No module named 'config'